In [116]:
import importlib

def install_if_not_installed(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} is not installed. Installing...")
        !pip install {package}
        print(f"{package} has been successfully installed.")

# Check and install ucimlrepo if not installed
install_if_not_installed("ucimlrepo")

# Import and use ucimlrepo
import ucimlrepo

ucimlrepo is already installed.


In [117]:
# import other necessary modules

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from ucimlrepo import fetch_ucirepo

import warnings     # filter warning messages
warnings.simplefilter(action="ignore")

In [118]:
# Define sigmoid, gradient descent and loss functions for LogReg

def sigmoid(z):
    """
    Sigmoid function to convert linear combination to probabilities.
    """
    return 1 / (1 + np.exp(-z))

def cross_entropy_loss(y_true, y_pred_prob):
    """
    Compute the cross-entropy loss.
    """
    return -np.mean(y_true * np.log(y_pred_prob) + (1 - y_true) * np.log(1 - y_pred_prob))

def gradient_descent_logistic(X, y, alpha, num_iterations):
    """
    Perform gradient descent to minimize the cross-entropy loss.
    """
    num_samples, num_features = X.shape
    theta = np.zeros(num_features + 1)  # Initialize model parameters to zero, including intercept

    # Insert a column of ones at the beginning of X (intercept term)
    X_with_intercept = np.hstack([np.ones((num_samples, 1)), X])

    # fixed number of iterations
    for _ in range(num_iterations):
        # Calculate predictions
        y_pred_prob = sigmoid(X_with_intercept @ theta)

        # Calculate gradients
        gradients = -(1 / num_samples) * X_with_intercept.T @ (y - y_pred_prob)

        # Update parameters
        theta -= alpha * gradients

    return theta

In [119]:
# Load data into DataFrame
spambase = fetch_ucirepo(id=94)
X = pd.DataFrame(spambase.data.features)
y = pd.DataFrame(spambase.data.targets)

# Normalize the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_normalized_values = scaler.fit_transform(X)
X = pd.DataFrame(X_normalized_values, columns=X.columns)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [120]:
# Define learning rates and number of iterations
learning_rates = [0.01, 0.1, 0.5]
num_iterations_list = [10, 50, 100]

# Initialize results dictionary
results = {'Learning rate': [], 'Num_iterations': [],
           'Accuracy': [], 'Precision': [],
           'Recall': [], 'F1 score': []}

# Iterate over learning rates and number of iterations
for alpha in learning_rates:
    for num_iterations in num_iterations_list:
        # Perform gradient descent
        theta = gradient_descent_logistic(X_train, y_train.values.ravel(),
                                          alpha, num_iterations)

        # Predict probabilities on the test set
        y_test_pred_prob = sigmoid(np.hstack([np.ones((len(X_test), 1)), X_test.values]) @ theta)
        y_test_pred = (y_test_pred_prob >= 0.5).astype(int)

        # Calculate evaluation metrics for testing set
        accuracy_test = accuracy_score(y_test, y_test_pred)
        precision_test = precision_score(y_test, y_test_pred)
        recall_test = recall_score(y_test, y_test_pred)
        f1_test = f1_score(y_test, y_test_pred)

        # Append results to the dictionary
        results['Learning rate'].append(alpha)
        results['Num_iterations'].append(num_iterations)
        results['Accuracy'].append(accuracy_test)
        results['Precision'].append(precision_test)
        results['Recall'].append(recall_test)
        results['F1 score'].append(f1_test)

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results)

# Display the results
print(results_df)

   Learning rate  Num_iterations  Accuracy  Precision    Recall  F1 score
0           0.01              10  0.903562   0.890558  0.873684  0.882040
1           0.01              50  0.905300   0.899563  0.867368  0.883173
2           0.01             100  0.901825   0.898678  0.858947  0.878364
3           0.10              10  0.901825   0.898678  0.858947  0.878364
4           0.10              50  0.904431   0.919540  0.842105  0.879121
5           0.10             100  0.908775   0.926267  0.846316  0.884488
6           0.50              10  0.906169   0.919908  0.846316  0.881579
7           0.50              50  0.912250   0.934884  0.846316  0.888398
8           0.50             100  0.916594   0.939675  0.852632  0.894040


In [121]:
# Initialize results dictionary
cross_entropy_results = {'Learning rate': [], 'Num_iterations': [],
           'CE Loss 10': [], 'CE Loss 50': [], 'CE Loss 100': []}

# Iterate over learning rates and number of iterations
for alpha in learning_rates:
    for num_iterations in num_iterations_list:
        # Perform gradient descent
        theta = gradient_descent_logistic(X_train, y_train.values.ravel(),
                                          alpha, num_iterations)

        # Calculate cross-entropy loss at different iterations
        theta_10 = gradient_descent_logistic(X_train, y_train.values.ravel(), alpha, 10)
        theta_50 = gradient_descent_logistic(X_train, y_train.values.ravel(), alpha, 50)
        theta_100 = gradient_descent_logistic(X_train, y_train.values.ravel(), alpha, 100)

        # Calculate cross-entropy loss at different iterations
        y_train_pred_prob_10 = sigmoid(np.hstack([np.ones((len(X_train), 1)), X_train.values]) @ theta_10)
        y_train_pred_prob_50 = sigmoid(np.hstack([np.ones((len(X_train), 1)), X_train.values]) @ theta_50)
        y_train_pred_prob_100 = sigmoid(np.hstack([np.ones((len(X_train), 1)), X_train.values]) @ theta_100)

        loss_10 = np.mean([cross_entropy_loss(y_train.iloc[i], y_train_pred_prob_10[i]) for i in range(len(X_train))])
        loss_50 = np.mean([cross_entropy_loss(y_train.iloc[i], y_train_pred_prob_50[i]) for i in range(len(X_train))])
        loss_100 = np.mean([cross_entropy_loss(y_train.iloc[i], y_train_pred_prob_100[i]) for i in range(len(X_train))])

        # Append results to the dictionary
        cross_entropy_results['Learning rate'].append(alpha)
        cross_entropy_results['Num_iterations'].append(num_iterations)
        cross_entropy_results['CE Loss 10'].append(loss_10)
        cross_entropy_results['CE Loss 50'].append(loss_50)
        cross_entropy_results['CE Loss 100'].append(loss_100)


# Create a DataFrame from the results dictionary
cross_entropy_results_df = pd.DataFrame(cross_entropy_results)

# Display the results
print(cross_entropy_results_df)


   Learning rate  Num_iterations  CE Loss 10  CE Loss 50  CE Loss 100
0           0.01              10    0.651361    0.541717     0.468710
1           0.01              50    0.651361    0.541717     0.468710
2           0.01             100    0.651361    0.541717     0.468710
3           0.10              10    0.465348    0.324753     0.290112
4           0.10              50    0.465348    0.324753     0.290112
5           0.10             100    0.465348    0.324753     0.290112
6           0.50              10    0.320919    0.259935     0.244397
7           0.50              50    0.320919    0.259935     0.244397
8           0.50             100    0.320919    0.259935     0.244397
